## Reading File Data to DataFrame

In [1]:
#import module
from pyspark.sql import SparkSession
from pyspark.sql.types import *

#create session in order to be capable of accessing all Spark API
spark = SparkSession \
    .builder \
    .appName("Introdution to Spark DataFrame") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

#define data schema for file we want to read
purchaseSchema = StructType([
    StructField("Date", DateType(), True),
    StructField("Time", StringType(), True),
    StructField("City", StringType(), True),
    StructField("Item", StringType(), True),
    StructField("Total", FloatType(), True),
    StructField("Payment", StringType(), True),
])    

# read csv file with our defined schema into Spark DataFrame, and use "tab" delimiter
purchaseDataframe = spark.read.csv(
    "dataset/purchases.csv", 
    header=True, schema=purchaseSchema, sep="\t")
#show 3 rows of our DataFrame
purchaseDataframe.show(3)

+----------+-----+----------+----------------+------+-------+
|      Date| Time|      City|            Item| Total|Payment|
+----------+-----+----------+----------------+------+-------+
|2012-01-01|09:00|  San Jose|  Men's Clothing|214.05|   Amex|
|2012-01-01|09:00|Fort Worth|Women's Clothing|153.57|   Visa|
|2012-01-01|09:00| San Diego|           Music| 66.08|   Cash|
+----------+-----+----------+----------------+------+-------+
only showing top 3 rows



## Counting number of rows, printing dataframe schema and printing statistic of our data

In [2]:
#count number of rows of our dataFrame
num_rows = purchaseDataframe.count()
print("number of rows: ", num_rows)
#show our dataFrame schema
purchaseDataframe.printSchema()
#show statistic of the data we want
purchaseDataframe.describe('Total').show()

number of rows:  4138476
root
 |-- Date: date (nullable = true)
 |-- Time: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Item: string (nullable = true)
 |-- Total: float (nullable = true)
 |-- Payment: string (nullable = true)

+-------+------------------+
|summary|             Total|
+-------+------------------+
|  count|           4138476|
|   mean|249.96108549398525|
| stddev| 144.3174111542959|
|    min|               0.0|
|    max|            499.99|
+-------+------------------+



## Creating a new dataFrame from a subset of our existing dataFrame

In [3]:
#create new dataFrame from "City" and "Total" columns
newDataframe = purchaseDataframe.select(purchaseDataframe['City'], 
                                              purchaseDataframe['Total'])
newDataframe.show(3); #menampilkan 3 baris DataFrame baru kita
newDataframe.printSchema() #print skema dari DataFrame baru kita

+----------+------+
|      City| Total|
+----------+------+
|  San Jose|214.05|
|Fort Worth|153.57|
| San Diego| 66.08|
+----------+------+
only showing top 3 rows

root
 |-- City: string (nullable = true)
 |-- Total: float (nullable = true)



## Adding a constant value to every row data in certain column

In [4]:
#adding constant value of 10 to every row data in "Total" column
purchaseDataframe.select(purchaseDataframe['City'],
                         purchaseDataframe['Total']+10).show(3)

+----------+------------+
|      City|(Total + 10)|
+----------+------------+
|  San Jose|      224.05|
|Fort Worth|      163.57|
| San Diego|       76.08|
+----------+------------+
only showing top 3 rows



## Filtering dataFrame using our defined condition

In [5]:
#filter only row data whose "Total" column value > 200
purchaseDataframe.filter(purchaseDataframe['Total'] > 200).show(3)

+----------+-----+----------+-------------------+------+----------+
|      Date| Time|      City|               Item| Total|   Payment|
+----------+-----+----------+-------------------+------+----------+
|2012-01-01|09:00|  San Jose|     Men's Clothing|214.05|      Amex|
|2012-01-01|09:00|Pittsburgh|       Pet Supplies|493.51|  Discover|
|2012-01-01|09:00|     Omaha|Children's Clothing|235.63|MasterCard|
+----------+-----+----------+-------------------+------+----------+
only showing top 3 rows



## Sorting dataFrame by certain column

In [6]:
sortedByCity = purchaseDataframe.orderBy('City').show(4)

+----------+-----+-----------+----------------+------+----------+
|      Date| Time|       City|            Item| Total|   Payment|
+----------+-----+-----------+----------------+------+----------+
|2012-10-07|11:11|Albuquerque|    Pet Supplies| 308.7|      Visa|
|2012-10-07|11:40|Albuquerque|            Toys|299.63|MasterCard|
|2012-10-07|11:13|Albuquerque|Women's Clothing|419.49|  Discover|
|2012-10-07|10:39|Albuquerque|    Pet Supplies| 401.3|MasterCard|
+----------+-----+-----------+----------------+------+----------+
only showing top 4 rows



## Calculating number of transactions in each city

In [7]:
numTransactionEachCity = purchaseDataframe.groupBy("City").count()
numTransactionEachCity.show(5)

+---------------+-----+
|           City|count|
+---------------+-----+
|North Las Vegas|40013|
|        Phoenix|40333|
|          Omaha|40209|
|      Anchorage|39806|
|        Anaheim|40086|
+---------------+-----+
only showing top 5 rows



## Indexing and Accessing DataFrame
Since Spark dataFrame is distributed into clusters, we cannot access it by [row,column] as in pandas dataFrame for example. There is an alternative way to do that by creating new column with "incremental ID". Then, we can access by row using ".filter()" function to our "incremental ID" column. 

In [8]:
#import monotonically_increasing_id
from pyspark.sql.functions import monotonically_increasing_id

newPurchasedDataframe = purchaseDataframe.withColumn(
    "index", monotonically_increasing_id())
newPurchasedDataframe.show(7)
row2Till4 = newPurchasedDataframe.filter((newPurchasedDataframe['index']>=2) &
                                         (newPurchasedDataframe['index']<=4))
row2Till4.show()

+----------+-----+----------+-------------------+------+----------+-----+
|      Date| Time|      City|               Item| Total|   Payment|index|
+----------+-----+----------+-------------------+------+----------+-----+
|2012-01-01|09:00|  San Jose|     Men's Clothing|214.05|      Amex|    0|
|2012-01-01|09:00|Fort Worth|   Women's Clothing|153.57|      Visa|    1|
|2012-01-01|09:00| San Diego|              Music| 66.08|      Cash|    2|
|2012-01-01|09:00|Pittsburgh|       Pet Supplies|493.51|  Discover|    3|
|2012-01-01|09:00|     Omaha|Children's Clothing|235.63|MasterCard|    4|
|2012-01-01|09:00|  Stockton|     Men's Clothing|247.18|MasterCard|    5|
|2012-01-01|09:00|    Austin|            Cameras| 379.6|      Visa|    6|
+----------+-----+----------+-------------------+------+----------+-----+
only showing top 7 rows

+----------+-----+----------+-------------------+------+----------+-----+
|      Date| Time|      City|               Item| Total|   Payment|index|
+----------+-

Then, to access it by row and column, use ".select()" function we ever used above before.

In [9]:
dataRow2ColumnTotal = newPurchasedDataframe.filter(newPurchasedDataframe['index']==2).select('Total')
dataRow2ColumnTotal.show()

+-----+
|Total|
+-----+
|66.08|
+-----+



# Using SQL query in dataFrame

Create a new dataFrame from the subset of our existing dataFrame

In [10]:
#we need to make sql temporary view for our dataFrame
purchaseDataframe.createOrReplaceTempView("purchaseSql")

#select "Total" dan "Payment" column from our sql temporary view
anotherNewDataframe = spark.sql("SELECT Total, Payment FROM purchaseSql")
anotherNewDataframe.show(3)

+------+-------+
| Total|Payment|
+------+-------+
|214.05|   Amex|
|153.57|   Visa|
| 66.08|   Cash|
+------+-------+
only showing top 3 rows



Sorting data by certain column

In [11]:
#sorting data by "City" column alphabetically
orderByCity = spark.sql("SELECT * FROM purchaseSql ORDER BY City")
orderByCity.show(5)

+----------+-----+-----------+----------------+------+----------+
|      Date| Time|       City|            Item| Total|   Payment|
+----------+-----+-----------+----------------+------+----------+
|2012-10-07|11:11|Albuquerque|    Pet Supplies| 308.7|      Visa|
|2012-10-07|11:41|Albuquerque|           Music|365.64|      Visa|
|2012-10-07|11:13|Albuquerque|Women's Clothing|419.49|  Discover|
|2012-10-07|10:39|Albuquerque|    Pet Supplies| 401.3|MasterCard|
|2012-10-07|11:18|Albuquerque|          Crafts|475.77|      Visa|
+----------+-----+-----------+----------------+------+----------+
only showing top 5 rows



The last one, we wil show how to filter data with "Total" column value > 200, and sort them by "Payment" column data.

In [12]:
#filter nilai kolom Total>50 dan urutkan berdasarkan cara pembayaran
filterAndSortWithSQL = spark.sql("SELECT * FROM purchaseSql WHERE Total>200 ORDER BY Payment")
filterAndSortWithSQL.show(4)

+----------+-----+--------------+-------------------+------+-------+
|      Date| Time|          City|               Item| Total|Payment|
+----------+-----+--------------+-------------------+------+-------+
|2012-10-07|10:34|      Richmond|Children's Clothing|252.45|   Amex|
|2012-10-07|10:36|San Bernardino|               Toys|272.91|   Amex|
|2012-10-07|10:34|     Baltimore|              Books|299.94|   Amex|
|2012-10-07|10:33|       Lincoln|       Pet Supplies|359.44|   Amex|
+----------+-----+--------------+-------------------+------+-------+
only showing top 4 rows

